In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

import scipy
import pylab
import math
import random

import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import NearestNeighbors

from fuzzywuzzy import fuzz

%matplotlib inline
sns.set()

C:\Users\skapr\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_excel(r'D:\Стас\Ученье - Свет\Оценка\Аспирантура\Видео по NN, Data mining, ABC и т.д\Распределение Гаусса, регрессия\Python\!Наработки\Квартиры Ектб_2\Выборка_ектб.xlsx', sheet_name = 0)
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Прогноз_1,Прогноз_2,MAE,MAPE,...,Нежилые помещения на первом этаже,строение,Квартира угловая / торцевая,Способ продажи,Отделка,Название новостройки,"Корпус, строение",Тип участия,Срок сдачи,Запланирован снос
0,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1_этажи,0,этажи,15.775605,7100000,7200000,7.357429e+06,7.315261e+06,-215260.780670,-0.030318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13340,8466_авито,1,авито,15.659230,6320000,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,переуступка,NaN,ЖК «Ботаника LIFE»,NaN,другое,Сдан 4 кв. 2023,NaN
13341,8467_авито,0,авито,15.147877,3790000,0,NaN,4.253817e+06,-463816.720736,-0.122379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13342,8468_авито,0,авито,15.054464,3452000,0,NaN,3.777030e+06,-325030.387942,-0.094157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13343,8469_авито,0,авито,15.176487,3900000,0,NaN,3.866094e+06,33905.795052,0.008694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[df['Мусор'] == 0]
df = df[df['Выбросы'] == 0]
#df = df[df['dup_res'] == 0]
df = df[df['Многоуровневость'] == 0]
df = df[df['Терраса'] == 0]
df.reset_index(drop = True , inplace = True )
df

,ID,Мусор,Агрегатор,LN(Цена),Цена,Цена_перв,Прогноз_1,Прогноз_2,MAE,MAPE,...,Нежилые помещения на первом этаже,строение,Квартира угловая / торцевая,Способ продажи,Отделка,Название новостройки,"Корпус, строение",Тип участия,Срок сдачи,Запланирован снос
0,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1_этажи,0,этажи,15.775605,7100000,7200000,7.357429e+06,7.315261e+06,-215260.780670,-0.030318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1_этажи,0,этажи,15.761421,7000000,7200000,7.357429e+06,7.315261e+06,-315260.780670,-0.045037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11236,8464_авито,0,авито,15.293700,4385000,0,5.291969e+06,5.297718e+06,-912718.329665,-0.208146,...,NaN,NaN,NaN,NaN,NaN,ЖК «Миллениум»,"Дом 5 (секц. А, Б)",другое,Сдан 2 кв. 2017,NaN
11237,8467_авито,0,авито,15.147877,3790000,0,NaN,4.253817e+06,-463816.720736,-0.122379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11238,8468_авито,0,авито,15.054464,3452000,0,NaN,3.777030e+06,-325030.387942,-0.094157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11239,8469_авито,0,авито,15.176487,3900000,0,NaN,3.866094e+06,33905.795052,0.008694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
list(df.columns)

['ID',
 'Мусор',
 'Агрегатор',
 'LN(Цена)',
 'Цена',
 'Цена_перв',
 'Прогноз_1',
 'Прогноз_2',
 'MAE',
 'MAPE',
 'Цена(кв.м)_пр',
 'Анализ',
 'Исп',
 'Комм',
 'Цена(кв.м)',
 'Выбросы',
 'Выбросы_1',
 'Исп.1',
 'Комм.1',
 'Выбросы_2',
 'Исп.2',
 'Комм.2',
 'IsolationForest',
 'pca',
 'COPOD',
 'dup_all',
 'dup_1',
 'dup_2',
 'Пересечения',
 'dup(<=0.03)',
 'Пересечения(<=0.03)',
 'Пересечения(>0.03)',
 'dup_res',
 'ОПлощадь',
 'Оплощадь_перв',
 'Комнатность',
 'Комнатность_перв',
 'Этаж_1',
 'Этаж_2',
 'Этаж_3',
 'Этаж_4',
 'Этаж/Этажность',
 'Этажность_1',
 'Этажность_2',
 'Неточности',
 'Переменная этажность',
 'Р',
 'Р%',
 'Минжкх',
 'П',
 'Правдом',
 'П.1',
 'Этажность_перв',
 'ГП',
 'Неточности.1',
 'Р.1',
 'Минжкх.1',
 'Правдом.1',
 'Год постройки',
 'Аварийные',
 'Исп.3',
 'Ремонт_1',
 'Ремонт_перв',
 'Санузел',
 'Стены_1',
 'Стены_перв',
 'Адрес_перв',
 'Адрес',
 'Проверка номера дома',
 'Дом',
 'Дом_Ya',
 'Проверка',
 'Адрес_yandex',
 'Долгота',
 'Широта',
 'Сцепка_к',
 'Сцепка

In [5]:
df_sample = df[[
                'Цена',            
                'ОПлощадь',
                'Этаж_1',
                'Долгота',
                'Широта',    
                'Адрес_yandex',
                'Ремонт_1',
                'dup_res'
              ]]
df_sample

,Цена,ОПлощадь,Этаж_1,Долгота,Широта,Адрес_yandex,Ремонт_1,dup_res
0,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0
1,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0
2,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0
3,7100000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0
4,7000000,70.0,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0
...,...,...,...,...,...,...,...,...
11236,4385000,31.0,13,60.572447,56.810960,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0
11237,3790000,38.1,9,60.649271,56.795727,"Россия, Свердловская область, Екатеринбург, Пр...",стандартный,0
11238,3452000,32.4,9,60.649271,56.795727,"Россия, Свердловская область, Екатеринбург, Пр...",стандартный,0
11239,3900000,32.8,6,60.649271,56.795727,"Россия, Свердловская область, Екатеринбург, Пр...",стандартный,0


In [6]:
le = LabelEncoder()
df_sample['Ремонт_1_enc'] = le.fit_transform(df_sample['Ремонт_1'])

C:\Users\skapr\AppData\Local\Temp\ipykernel_17100\1602058538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['Ремонт_1_enc'] = le.fit_transform(df_sample['Ремонт_1'])


In [7]:
numerical_cols = [
                  'Цена',            
                  'ОПлощадь',
                  'Этаж_1',
                  'Долгота',
                  'Широта',
                  'Ремонт_1_enc'
                 ]

In [8]:
scaler = StandardScaler()
df_sample_scale = scaler.fit_transform(df_sample[numerical_cols])
df_sample_scale

array([[-0.01430021,  0.67572273, -0.7317098 , -1.32450692, -0.28415822,
        -0.33628422],
       [-0.01430021,  0.67572273, -0.7317098 , -1.32450692, -0.28415822,
        -0.33628422],
       [-0.01430021,  0.67572273, -0.7317098 , -1.32450692, -0.28415822,
        -0.33628422],
       ...,
       [-0.72214773, -0.86939211,  0.19035775,  0.80026397, -0.627172  ,
        -0.33628422],
       [-0.63276901, -0.85308483, -0.27067603,  0.80026397, -0.627172  ,
        -0.33628422],
       [-0.63276901, -0.55955378, -0.1169981 ,  0.80026397, -0.627172  ,
        -0.33628422]])

In [9]:
# Поиск ближайших соседей
k = 10  # Количество ближайших соседей
nbrs = NearestNeighbors(n_neighbors = k, 
                        algorithm = 'ball_tree',
                        metric = 'minkowski'  # metric = 'cosine', 'euclidean', 'minkowski', 'cityblock' (manhattan), 'manhattan', 'haversine' 
                       )                      # ball_tree эффективнее для средних размеров данных

nbrs.fit(df_sample_scale)

distances, indices = nbrs.kneighbors(df_sample_scale) # получаем дистанции и их индексы по k ближайшим соседям для каждого элемента таблицы

# Складываем дистанции и их индексы (списки) в отдельные столбцы
distance = []
indice = []

for dist, idx in zip(distances, indices):
    distance.append(dist)
    indice.append(idx)

df_sample['distances'] = distance
df_sample['indices'] = indice

# Фильтрация по порогу
distance_threshold = 0.1 # Порог расстояния (настраиваемый параметр)
address_threshold = 98   # Порог Fuzzy Matching для адресов (настраиваемый параметр)

duplicates = [False] * len(df_sample_scale) # создаем список дупликатов с меткой False
original = [False] * len(df_sample_scale)   # создаем список оригиналов с меткой False

for i, (dist, idx) in tqdm(enumerate(zip(distances, indices)), total = df_sample_scale.shape[0], position = 0):
    close_neighbors = []
    idx_new = [x for x in idx if x != i] # убираем индекс самого себя
    dist_idx = [j for j, x in enumerate(idx) if x == i][0] # находим индекс самого себя в списке dist
    dist = list(dist) # преобразуем dist array в list
    dist.pop(dist_idx) # исключаем дистанцию самого себя из dist
    for j, neighbor_index in enumerate(idx_new):
        if dist[j] <= distance_threshold: # если дистанция меньше порога
            close_neighbors.append(neighbor_index) # то складываем ее индекс в список для дальнейшего сравнения адресов

    if close_neighbors: # если список не пустой 
        for neighbor_index in close_neighbors: # то пиребераем его и сравниваем адреса
            if (
                not original[neighbor_index] and # если объект не находится в оригиналах
                not duplicates[neighbor_index] and # и если объект не находится в дубликатах и адреса совпадают
                fuzz.token_set_ratio(df_sample['Адрес_yandex'][i], df_sample['Адрес_yandex'][neighbor_index]) > address_threshold
               ):
                
                if not original[i]: # зесли наш объект не в оригиналах, то отправляем его туда
                    original[i] = True
                duplicates[neighbor_index] = True # закидываем данный объект в дупликаты

#duplicates = np.where(original, False, duplicates)
df_sample['duplicates'] = duplicates
df_sample

C:\Users\skapr\AppData\Local\Temp\ipykernel_17100\496132168.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['distances'] = distance
C:\Users\skapr\AppData\Local\Temp\ipykernel_17100\496132168.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['indices'] = indice
100%|█████████████████████████████████████████████████████████████████████████| 11241/11241 [00:00<00:00, 22858.92it/s]
C:\Users\skapr\AppData\Local\Temp\ipykernel_17100\496132168.py:53: SettingWithCopyWarning: 
A value is

,Цена,ОПлощадь,Этаж_1,Долгота,Широта,Адрес_yandex,Ремонт_1,dup_res,Ремонт_1_enc,distances,indices,duplicates
0,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[0, 9, 1, 6, 2, 7, 4, 3, 8, 5]",False
1,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[0, 9, 1, 6, 2, 7, 4, 3, 8, 5]",True
2,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[0, 9, 1, 6, 2, 7, 4, 3, 8, 5]",True
3,7100000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.008153640305817667, 0.0199506064693655...","[3, 5, 0, 6, 2, 1, 9, 7, 4, 8]",True
4,7000000,70.0,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.012230460458726222, 0.0122304604587262...","[4, 0, 9, 6, 1, 2, 7, 5, 8, 3]",True
...,...,...,...,...,...,...,...,...,...,...,...,...
11236,4385000,31.0,13,60.572447,56.810960,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.4193250781887334, 0.5530419868410641, ...","[11236, 10509, 6367, 2506, 1629, 4059, 3775, 1...",False
11237,3790000,38.1,9,60.649271,56.795727,"Россия, Свердловская область, Екатеринбург, Пр...",стандартный,0,3,"[0.0, 0.24196507820140717, 0.3088651831398408,...","[11237, 11238, 1034, 11240, 3586, 4341, 2345, ...",False
11238,3452000,32.4,9,60.649271,56.795727,"Россия, Свердловская область, Екатеринбург, Пр...",стандартный,0,3,"[0.0, 0.16023394754590922, 0.24196507820140717...","[11238, 1034, 11237, 2415, 9327, 11240, 7951, ...",False
11239,3900000,32.8,6,60.649271,56.795727,"Россия, Свердловская область, Екатеринбург, Пр...",стандартный,0,3,"[0.0, 0.3171209096195443, 0.3313267005444827, ...","[11239, 1034, 11240, 665, 7157, 7184, 6439, 23...",False


In [185]:
h = 1

dist = distances[h]
idx = indices[h]
i = h

# duplicates = [False] * len(df_sample_scale) # создаем список дупликатов с меткой False
# original = [False] * len(df_sample_scale)   # создаем список оригиналов с меткой False

close_neighbors = []
idx_new = [x for x in idx if x != i]
dist_idx = [j for j, x in enumerate(idx) if x == i][0]
dist = list(dist)
dist.pop(dist_idx)
for j, neighbor_index in enumerate(idx_new):
    if dist[j] <= distance_threshold:
        close_neighbors.append(neighbor_index)

if close_neighbors: # если список не пустой 
    for neighbor_index in close_neighbors: # то перебираем его и сравниваем адреса
        if (
            not original[neighbor_index] and # если объект не находится в оригиналах
            not duplicates[neighbor_index] and # и если объект не находится в дубликатах и адреса совпадают
            fuzz.token_set_ratio(df_sample['Адрес_yandex'][h], df_sample['Адрес_yandex'][neighbor_index]) > address_threshold
           ):
            
            if not original[h]: # зесли наш объект не в оригиналах, то отправляем его туда
                original[h] = True
            duplicates[neighbor_index] = True # закидываем данный объект в дупликаты

In [180]:
dist

[0.0,
 0.0,
 0.0,
 0.0,
 0.0015963040963787776,
 0.012230460458726222,
 0.01995060646936552,
 0.01995060646936552,
 0.0215524603869754]

In [181]:
close_neighbors

[2, 0, 9, 6, 7, 4, 8, 3, 5]

In [182]:
idx_new

[2, 0, 9, 6, 7, 4, 8, 3, 5]

In [144]:
df_sample.head(10)

,Цена,ОПлощадь,Этаж_1,Долгота,Широта,Адрес_yandex,dup_res,distances,indices,duplicates
0,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[2, 0, 1, 9, 6, 7, 4, 8, 3, 5]",False
1,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[2, 0, 1, 9, 6, 7, 4, 8, 3, 5]",True
2,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[2, 0, 1, 9, 6, 7, 4, 8, 3, 5]",True
3,7100000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.008153640305817667, 0.0199506064693655...","[3, 5, 0, 9, 6, 1, 2, 7, 4, 8]",True
4,7000000,70.0,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.012230460458726222, 0.0122304604587262...","[4, 6, 0, 2, 9, 1, 7, 5, 3, 8]",True
5,7100000,70.1,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.008153640305817667, 0.0203628868546346...","[5, 3, 4, 0, 2, 6, 1, 9, 7, 8]",True
6,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, улица Соболева, ...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[2, 0, 1, 9, 6, 7, 4, 8, 3, 5]",True
7,7000000,70.3,3,60.514631,56.810472,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.0015963040963787776, 0.001596304096378...","[7, 2, 0, 9, 6, 1, 4, 3, 8, 5]",True
8,6900000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.01995060646936552, 0.01995060646936552...","[8, 2, 0, 6, 9, 1, 7, 4, 3, 5]",True
9,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[2, 0, 1, 9, 6, 7, 4, 8, 3, 5]",True


In [122]:
df_sample['Адрес_yandex'].head(10).values

array(['Россия, Свердловская область, Екатеринбург, улица Соболева, 19',
       'Россия, Свердловская область, Екатеринбург, улица Соболева, 19',
       'Россия, Свердловская область, Екатеринбург, улица Соболева,',
       'Россия, Свердловская область, Екатеринбург, улица Соболева, 19',
       'Россия, Свердловская область, Екатеринбург, улица Соболева, 19',
       'Россия, Свердловская область, Екатеринбург, улица Соболева,',
       'Россия, Свердловская область, улица Соболева, 19, Екатеринбург',
       'Россия, Свердловская область, Екатеринбург, улица Соболева, 19',
       'Россия, Свердловская область, Екатеринбург, улица Соболева, ',
       'Россия, Свердловская область, Екатеринбург, улица Соболева, 19'],
      dtype=object)

In [10]:
dup = df_sample[df_sample['dup_res'] == 1]
dup

,Цена,ОПлощадь,Этаж_1,Долгота,Широта,Адрес_yandex,Ремонт_1,dup_res,Ремонт_1_enc,distances,indices,duplicates
10,11200000,78.0,21,60.590117,56.804397,"Россия, Свердловская область, Екатеринбург, ул...",без_отделки,1,0,"[0.0, 0.4619863947639256, 0.7311905785332116, ...","[10, 1794, 3472, 3276, 3269, 3275, 3277, 3471,...",False
12,5700000,30.4,1,60.600601,56.908989,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,1,3,"[0.0, 0.0, 0.42155634600418246, 0.421556346004...","[12, 10224, 4174, 4202, 7061, 5944, 4097, 1617...",False
13,2100000,23.0,2,60.583910,56.907082,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,1,3,"[0.0, 0.07587333302703528, 0.17388714931681942...","[13, 8494, 8083, 7790, 7771, 1366, 2058, 4591,...",False
16,7500000,36.0,20,60.608111,56.866854,"Россия, Свердловская область, Екатеринбург, пр...",улучшенный,1,5,"[0.0, 0.0, 0.39075552249198636, 0.558089648794...","[6533, 16, 5475, 1041, 6167, 7050, 1275, 998, ...",False
17,4500000,30.2,3,60.583712,56.833092,"Россия, Свердловская область, Екатеринбург, Мо...",стандартный,1,3,"[0.0, 0.0, 0.20051227538339783, 0.206258569351...","[17, 6027, 2376, 1990, 4849, 5584, 2409, 6178,...",False
...,...,...,...,...,...,...,...,...,...,...,...,...
11188,4589200,29.8,20,60.587692,56.759646,"Россия, Свердловская область, Екатеринбург, 2-...",стандартный,1,3,"[0.0, 0.0, 0.3021306507417921, 0.3037446089565...","[11188, 11190, 11183, 9784, 1586, 10586, 11209...",False
11189,7539000,47.6,26,60.564632,56.804451,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,1,3,"[0.0, 0.15369980884241793, 0.6229127856333804,...","[11189, 11171, 11185, 11187, 10918, 11179, 266...",False
11193,8085000,47.4,31,60.564632,56.804451,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,1,3,"[0.0, 0.07530701907382231, 0.14814231843673414...","[11193, 11173, 11202, 11187, 11179, 10943, 109...",True
11197,5080800,43.8,2,60.524900,56.872423,"Россия, Свердловская область, Екатеринбург, Ко...",требует_косметического_ремонта,1,4,"[0.0, 0.07135754247074766, 0.08162262480033887...","[11197, 10882, 10738, 10838, 10909, 10794, 108...",True


In [12]:
dup['Адрес_yandex'].loc[11207]

'Россия, Свердловская область, Екатеринбург, улица Академика Бардина, 26А'

In [11]:
df_sample[df_sample['duplicates'] == True]

,Цена,ОПлощадь,Этаж_1,Долгота,Широта,Адрес_yandex,Ремонт_1,dup_res,Ремонт_1_enc,distances,indices,duplicates
1,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[0, 9, 1, 6, 2, 7, 4, 3, 8, 5]",True
2,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[0, 9, 1, 6, 2, 7, 4, 3, 8, 5]",True
3,7100000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.008153640305817667, 0.0199506064693655...","[3, 5, 0, 6, 2, 1, 9, 7, 4, 8]",True
4,7000000,70.0,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.012230460458726222, 0.0122304604587262...","[4, 0, 9, 6, 1, 2, 7, 5, 8, 3]",True
5,7100000,70.1,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.008153640305817667, 0.0203628868546346...","[5, 3, 4, 9, 6, 0, 1, 2, 7, 8]",True
...,...,...,...,...,...,...,...,...,...,...,...,...
11219,7351459,63.4,2,60.524900,56.872423,"Россия, Свердловская область, Екатеринбург, Ко...",требует_косметического_ремонта,0,4,"[0.0, 0.05573431386308439, 0.16797262084718675...","[11219, 10613, 10699, 10830, 10874, 10689, 448...",True
11225,5707200,46.4,4,60.524900,56.872423,"Россия, Свердловская область, Екатеринбург, Ко...",требует_косметического_ремонта,0,4,"[0.0, 0.09023427738333151, 0.13799310610414525...","[11225, 10915, 10823, 10852, 10747, 10789, 109...",True
11230,9100839,68.7,9,60.496648,56.784792,"Россия, Свердловская область, Екатеринбург, ул...",стандартный,0,3,"[0.0, 0.028333598250962183, 0.0841303422675279...","[11230, 10914, 10936, 10921, 8270, 10571, 1056...",True
11231,5637300,43.7,7,60.524900,56.872423,"Россия, Свердловская область, Екатеринбург, Ко...",требует_косметического_ремонта,0,4,"[0.0, 0.004821205746369578, 0.0413198115716797...","[11231, 10899, 10869, 10893, 10833, 10850, 108...",True


In [195]:
dup['duplicates'].value_counts()

duplicates
False    2658
True      684
Name: count, dtype: int64

In [147]:
dup_2 = df_sample[df_sample['duplicates'] == True]
dup_2

,Цена,ОПлощадь,Этаж_1,Долгота,Широта,Адрес_yandex,dup_res,distances,indices,duplicates
1,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[2, 0, 1, 9, 6, 7, 4, 8, 3, 5]",True
2,7000000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.001596304096378777...","[2, 0, 1, 9, 6, 7, 4, 8, 3, 5]",True
3,7100000,70.3,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.008153640305817667, 0.0199506064693655...","[3, 5, 0, 9, 6, 1, 2, 7, 4, 8]",True
4,7000000,70.0,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.012230460458726222, 0.0122304604587262...","[4, 6, 0, 2, 9, 1, 7, 5, 3, 8]",True
5,7100000,70.1,3,60.514731,56.810482,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.008153640305817667, 0.0203628868546346...","[5, 3, 4, 0, 2, 6, 1, 9, 7, 8]",True
...,...,...,...,...,...,...,...,...,...,...
11226,9444000,78.7,23,60.587692,56.759646,"Россия, Свердловская область, Екатеринбург, 2-...",0,"[0.0, 0.009944598757109718, 0.1543085678601806...","[11226, 3355, 11215, 3354, 11170, 10923, 3353,...",True
11227,9349000,60.4,25,60.564632,56.804451,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.042920309058283736, 0.0570754821407228...","[11227, 10157, 9840, 3264, 10918, 3265, 6259, ...",True
11230,9100839,68.7,9,60.496648,56.784792,"Россия, Свердловская область, Екатеринбург, ул...",0,"[0.0, 0.011378193013079655, 0.0283335982509621...","[11230, 3530, 10914, 10936, 10921, 8270, 10571...",True
11231,5637300,43.7,7,60.524900,56.872423,"Россия, Свердловская область, Екатеринбург, Ко...",0,"[0.0, 0.004821205746369578, 0.0413198115716797...","[11231, 10899, 10869, 10893, 10833, 10850, 108...",True


In [148]:
dup_2.iloc[10:20]

,Цена,ОПлощадь,Этаж_1,Долгота,Широта,Адрес_yandex,dup_res,distances,indices,duplicates
85,5999000,44.4,5,60.652954,56.791093,"Россия, Свердловская область, Екатеринбург, Па...",0,"[0.0, 0.0, 0.0, 0.19916824711564735, 0.1991682...","[85, 83, 84, 247, 4640, 11115, 723, 11137, 111...",True
615,5300000,43.5,2,60.576032,56.800232,"Россия, Свердловская область, Екатеринбург, ул...",1,"[0.0, 0.003990121293873106, 0.0799443460099460...","[615, 9243, 57, 10421, 7320, 7679, 9944, 9938,...",True
747,3200000,25.2,15,60.490386,56.790989,"Россия, Свердловская область, Екатеринбург, ул...",1,"[0.0, 0.0, 0.0, 0.169322189511102, 0.175774949...","[8650, 101, 747, 8579, 2778, 10972, 2590, 1096...",True
866,2700000,20.0,17,60.696316,56.780803,"Россия, Свердловская область, Екатеринбург, ми...",1,"[0.0, 0.0, 0.01630728061163511, 0.019950606469...","[9459, 866, 8090, 10419, 691, 9179, 1962, 9241...",True
870,4300000,44.0,3,60.610509,56.908178,"Россия, Свердловская область, Екатеринбург, Че...",1,"[0.0, 0.0, 0.07338276275235794, 0.073382762752...","[870, 7517, 166, 10091, 9107, 7310, 7271, 1005...",True
989,6100000,54.0,12,60.534359,56.789185,"Россия, Свердловская область, Екатеринбург, ул...",1,"[0.0, 0.02881533850814218, 0.04987651617341382...","[989, 8383, 784, 9196, 10254, 7331, 7272, 9854...",True
996,3200000,25.2,17,60.766537,56.799621,"Россия, Свердловская область, Екатеринбург, Ла...",1,"[0.0, 0.032614561223270444, 0.0326145612232704...","[996, 988, 11059, 10441, 1872, 5965, 8279, 466...",True
1038,5700000,44.5,3,60.525160,56.769543,"Россия, Свердловская область, Екатеринбург, ул...",1,"[0.0, 0.0, 0.057045184283147636, 0.16407002809...","[1038, 6540, 81, 5124, 5258, 2053, 4715, 2809,...",True
1046,6600000,74.0,9,60.819331,56.755442,"Россия, Свердловская область, Екатеринбург, ул...",1,"[0.0, 0.05985181940809657, 0.05985181940809657...","[1046, 6149, 1027, 5606, 6451, 9058, 644, 4945...",True
1098,3700000,33.0,9,60.699325,56.778875,"Россия, Свердловская область, Екатеринбург, ми...",0,"[0.0, 0.009975303234682764, 0.1295432875470882...","[1098, 79, 6480, 6195, 5612, 6160, 8625, 1841,...",True


In [128]:
dup_2.iloc[10:20]['Адрес_yandex'].values

array(['Россия, Свердловская область, Екатеринбург, Павлодарская улица, 2/1',
       'Россия, Свердловская область, Екатеринбург, улица Академика Бардина, 44',
       'Россия, Свердловская область, Екатеринбург, улица Евгения Савкова, 19А',
       'Россия, Свердловская область, Екатеринбург, микрорайон Светлый, 11',
       'Россия, Свердловская область, Екатеринбург, Черноярская улица, 10А',
       'Россия, Свердловская область, Екатеринбург, улица Волошина, 2',
       'Россия, Свердловская область, Екатеринбург, Латвийская улица, 58',
       'Россия, Свердловская область, Екатеринбург, улица Академика Ландау, 9',
       'Россия, Свердловская область, Екатеринбург, улица Бахчиванджи, 15',
       'Россия, Свердловская область, Екатеринбург, микрорайон Светлый, 1'],
      dtype=object)

In [ ]:
#df_cleaned = df[~df['дубликат']] #Удаляем дубликаты из датафрейма

Обратите внимание, что kd_tree эффективен для низкоразмерных данных.  Для высокоразмерных данных лучше использовать ball_tree или другие структуры данных, предназначенные для больших размерностей.  Если у вас много признаков,  возможно, стоит рассмотреть методы понижения размерности (PCA, t-SNE)  перед поиском ближайших соседей.  Это может улучшить как производительность, так и качество результатов.  Кроме того,  выбор между manhattan и euclidean зависит от вашей задачи и  характера данных.  Эксперименты на ваших данных помогут определить, какой метод работает лучше.

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

data = {
    'площадь': [50, 50.1, 60, 50.2, 75, 50.05, 49.9, 50.08, 50.12],
    'этаж': [3, 3, 2, 3, 1, 3, 3, 3, 3],
    'долгота': [55.34, 55.35, 55.36, 55.345, 56.1, 55.342, 55.338, 55.348, 55.352],
    'широта': [37.93, 37.93, 37.94, 37.93, 37.92, 37.931, 37.929, 37.933, 37.935],
    'цена': [8000, 8100, 9000, 8050, 10000, 8020, 7980, 8080, 8120],
    'адрес': ['ул. Ленина 10', 'ул. Ленина, 10 кв.1', 'ул. Кирова 5', 'ул. Ленина 10', 'ул. Мира 2', 'ул. Ленина 10А', 'ул. Ленина 10Б', 'ул. Ленина 10В', 'ул. Ленина 10Г']
}

df = pd.DataFrame(data)

# Нормализация числовых данных
numerical_cols = ['площадь', 'этаж', 'долгота', 'широта', 'цена']
df_normalized = (df[numerical_cols] - df[numerical_cols].mean()) / df[numerical_cols].std()
df_normalized = df_normalized.fillna(0)


# Пороговое значение для расстояния (настраиваемый параметр)
distance_threshold = 0.5

# Вычисляем попарные расстояния
distances = np.sqrt(np.sum((df_normalized - df_normalized[:, np.newaxis])**2, axis=2))

# Создаем матрицу сходства
similarity_matrix = np.where(distances <= distance_threshold, 1, 0)

# Применение fuzzywuzzy к адресам и корректировка матрицы
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        if similarity_matrix[i, j] == 1 and fuzz.ratio(df['адрес'][i], df['адрес'][j]) >= 80:
            similarity_matrix[i, j] = 2
            similarity_matrix[j, i] = 2

# Определение дубликатов на основе матрицы
df['дубликат'] = np.sum(similarity_matrix, axis=0) > 1

print(df)